# Лабораторная работа №3. Решающее дерево

**Цель:** Исследование и реализация решающего дерева.


In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor

from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    mean_absolute_error,
    mean_squared_error,
    r2_score
)

mush = pd.read_csv("data/mushrooms.csv")
cars = pd.read_csv("data/car_price.csv")

X_mush = mush.drop("class", axis=1)
y_mush = mush["class"]

X_cars = cars.drop("Price", axis=1)
y_cars = cars["Price"]

# разбиение
X_train_m, X_test_m, y_train_m, y_test_m = train_test_split(
    X_mush, y_mush, test_size=0.2, random_state=42, stratify=y_mush
)

X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(
    X_cars, y_cars, test_size=0.2, random_state=42
)

# препроцессоры
preprocess_mush = ColumnTransformer(
    transformers=[("cat", OneHotEncoder(handle_unknown="ignore"), X_mush.columns.tolist())]
)

preprocess_cars = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), X_cars.select_dtypes(include=["int64","float64"]).columns.tolist()),
        ("cat", OneHotEncoder(handle_unknown="ignore"), X_cars.select_dtypes(include=["object"]).columns.tolist())
    ]
)


In [3]:
dt_clf_baseline = Pipeline(steps=[
    ("preprocess", preprocess_mush),
    ("model", DecisionTreeClassifier(random_state=42))
])

dt_clf_baseline.fit(X_train_m, y_train_m)
y_pred_m_dt = dt_clf_baseline.predict(X_test_m)

print("=== Decision Tree Baseline (Mushrooms) ===")
print("Accuracy:", accuracy_score(y_test_m, y_pred_m_dt))
print("\nClassification report:")
print(classification_report(y_test_m, y_pred_m_dt))
print("\nConfusion matrix:")
print(confusion_matrix(y_test_m, y_pred_m_dt))


=== Decision Tree Baseline (Mushrooms) ===
Accuracy: 1.0

Classification report:
              precision    recall  f1-score   support

           e       1.00      1.00      1.00       842
           p       1.00      1.00      1.00       783

    accuracy                           1.00      1625
   macro avg       1.00      1.00      1.00      1625
weighted avg       1.00      1.00      1.00      1625


Confusion matrix:
[[842   0]
 [  0 783]]


In [4]:
dt_reg_baseline = Pipeline(steps=[
    ("preprocess", preprocess_cars),
    ("model", DecisionTreeRegressor(random_state=42))
])

dt_reg_baseline.fit(X_train_c, y_train_c)
y_pred_c_dt = dt_reg_baseline.predict(X_test_c)

mae_dt = mean_absolute_error(y_test_c, y_pred_c_dt)
rmse_dt = mean_squared_error(y_test_c, y_pred_c_dt)**0.5
r2_dt = r2_score(y_test_c, y_pred_c_dt)

print("=== Decision Tree Baseline (Cars) ===")
print("MAE:", mae_dt)
print("RMSE:", rmse_dt)
print("R^2:", r2_dt)


=== Decision Tree Baseline (Cars) ===
MAE: 2590.12241036684
RMSE: 3192.0472249858512
R^2: 0.6276799533493813


## 2. Бейзлайн (Sklearn)

1. **Mushrooms (Class):** Accuracy = 1.0. Идеально.
2. **Cars (Reg):** MAE ~2590, R2 ~0.63.
   Дерево без настройки переобучается и работает хуже линейной регрессии (R2 ~0.82).


In [5]:
# Улучшенное решающее дерево для грибов

dt_clf_pipe = Pipeline(steps=[
    ("preprocess", preprocess_mush),
    ("model", DecisionTreeClassifier(random_state=42))
])

param_grid_clf = {
    "model__max_depth": [None, 3, 5, 7, 10],
    "model__min_samples_split": [2, 5, 10],
    "model__min_samples_leaf": [1, 2, 5]
}

grid_dt_clf = GridSearchCV(
    estimator=dt_clf_pipe,
    param_grid=param_grid_clf,
    cv=5,
    scoring="f1_macro",
    n_jobs=-1
)

grid_dt_clf.fit(X_train_m, y_train_m)

print("Best params (DT Mushrooms):", grid_dt_clf.best_params_)

best_dt_clf = grid_dt_clf.best_estimator_

y_pred_m_dt_best = best_dt_clf.predict(X_test_m)

print("\n=== Improved Decision Tree (Mushrooms) ===")
print("Accuracy:", accuracy_score(y_test_m, y_pred_m_dt_best))
print("\nClassification report:")
print(classification_report(y_test_m, y_pred_m_dt_best))
print("\nConfusion matrix:")
print(confusion_matrix(y_test_m, y_pred_m_dt_best))


Best params (DT Mushrooms): {'model__max_depth': None, 'model__min_samples_leaf': 1, 'model__min_samples_split': 2}

=== Improved Decision Tree (Mushrooms) ===
Accuracy: 1.0

Classification report:
              precision    recall  f1-score   support

           e       1.00      1.00      1.00       842
           p       1.00      1.00      1.00       783

    accuracy                           1.00      1625
   macro avg       1.00      1.00      1.00      1625
weighted avg       1.00      1.00      1.00      1625


Confusion matrix:
[[842   0]
 [  0 783]]


In [6]:
# Улучшенное решающее дерево для машин

dt_reg_pipe = Pipeline(steps=[
    ("preprocess", preprocess_cars),
    ("model", DecisionTreeRegressor(random_state=42))
])

param_grid_reg = {
    "model__max_depth": [None, 3, 5, 7, 10],
    "model__min_samples_split": [2, 5, 10],
    "model__min_samples_leaf": [1, 2, 5]
}

grid_dt_reg = GridSearchCV(
    estimator=dt_reg_pipe,
    param_grid=param_grid_reg,
    cv=5,
    scoring="neg_mean_absolute_error",
    n_jobs=-1
)

grid_dt_reg.fit(X_train_c, y_train_c)

print("Best params (DT Cars):", grid_dt_reg.best_params_)

best_dt_reg = grid_dt_reg.best_estimator_

y_pred_c_dt_best = best_dt_reg.predict(X_test_c)

mae_dt_best = mean_absolute_error(y_test_c, y_pred_c_dt_best)
rmse_dt_best = mean_squared_error(y_test_c, y_pred_c_dt_best)**0.5
r2_dt_best = r2_score(y_test_c, y_pred_c_dt_best)

print("\n=== Improved Decision Tree (Cars) ===")
print("MAE:", mae_dt_best)
print("RMSE:", rmse_dt_best)
print("R^2:", r2_dt_best)


Best params (DT Cars): {'model__max_depth': 7, 'model__min_samples_leaf': 5, 'model__min_samples_split': 2}

=== Improved Decision Tree (Cars) ===
MAE: 2090.5980932957514
RMSE: 2633.0894896335567
R^2: 0.74665692151086


## 3. Улучшение (GridSearchCV)

1. **Mushrooms**: Без изменений (1.0).
2. **Cars**: Подбор глубины (`max_depth=7`) и размера листа улучшил R2 до ~0.75 (с 0.63). Переобучение уменьшилось.


In [7]:
# Закодированные данные для своих реализаций

X_train_m_enc = preprocess_mush.fit_transform(X_train_m)
X_test_m_enc = preprocess_mush.transform(X_test_m)

X_train_c_enc = preprocess_cars.fit_transform(X_train_c)
X_test_c_enc = preprocess_cars.transform(X_test_c)

print("Encoded mushrooms:", X_train_m_enc.shape, X_test_m_enc.shape)
print("Encoded cars:", X_train_c_enc.shape, X_test_c_enc.shape)

Encoded mushrooms: (6499, 117) (1625, 117)
Encoded cars: (800, 18) (200, 18)


In [8]:
import numpy as np

class TreeNode:
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, value=None):
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value  # значение в листе


class MyDecisionTreeBase:
    def __init__(self, max_depth=5, min_samples_split=2, min_samples_leaf=1):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.root = None

    def _to_dense(self, X):
        if hasattr(X, "toarray"):
            X = X.toarray()
        return np.array(X)

    def fit(self, X, y):
        X = self._to_dense(X)
        y = np.array(y)
        self.root = self._build_tree(X, y, depth=0)
        return self

    def _build_tree(self, X, y, depth):
        num_samples, num_features = X.shape

        if (depth >= self.max_depth or
            num_samples < self.min_samples_split or
            len(np.unique(y)) == 1):
            leaf_value = self._leaf_value(y)
            return TreeNode(value=leaf_value)

        best_feature, best_threshold = self._best_split(X, y)
        if best_feature is None:
            leaf_value = self._leaf_value(y)
            return TreeNode(value=leaf_value)

        left_mask = X[:, best_feature] <= best_threshold
        right_mask = ~left_mask

        if left_mask.sum() < self.min_samples_leaf or right_mask.sum() < self.min_samples_leaf:
            leaf_value = self._leaf_value(y)
            return TreeNode(value=leaf_value)

        left_child = self._build_tree(X[left_mask], y[left_mask], depth + 1)
        right_child = self._build_tree(X[right_mask], y[right_mask], depth + 1)

        return TreeNode(feature_index=best_feature,
                        threshold=best_threshold,
                        left=left_child,
                        right=right_child)

    def _best_split(self, X, y):
        num_samples, num_features = X.shape
        best_feature, best_threshold = None, None
        best_score = np.inf

        for feature_idx in range(num_features):
            values = X[:, feature_idx]
            unique_vals = np.unique(values)
            if unique_vals.size == 1:
                continue

            thresholds = (unique_vals[:-1] + unique_vals[1:]) / 2.0

            for thr in thresholds:
                left_mask = values <= thr
                right_mask = ~left_mask
                if left_mask.sum() == 0 or right_mask.sum() == 0:
                    continue

                score = self._split_criterion(y[left_mask], y[right_mask])
                if score < best_score:
                    best_score = score
                    best_feature = feature_idx
                    best_threshold = thr

        return best_feature, best_threshold

    def _leaf_value(self, y):
        raise NotImplementedError

    def _split_criterion(self, y_left, y_right):
        raise NotImplementedError

    def _predict_row(self, x, node):
        if node.value is not None:
            return node.value

        if x[node.feature_index] <= node.threshold:
            return self._predict_row(x, node.left)
        else:
            return self._predict_row(x, node.right)

    def predict(self, X):
        X = self._to_dense(X)
        preds = [self._predict_row(row, self.root) for row in X]
        return np.array(preds)


class MyDecisionTreeClassifier(MyDecisionTreeBase):
    def __init__(self, max_depth=5, min_samples_split=2, min_samples_leaf=1):
        super().__init__(max_depth, min_samples_split, min_samples_leaf)

    def _leaf_value(self, y):
        values, counts = np.unique(y, return_counts=True)
        return values[np.argmax(counts)]

    def _gini(self, y):
        _, counts = np.unique(y, return_counts=True)
        p = counts / counts.sum()
        return 1.0 - np.sum(p ** 2)

    def _split_criterion(self, y_left, y_right):
        n = len(y_left) + len(y_right)
        return (len(y_left) / n) * self._gini(y_left) + (len(y_right) / n) * self._gini(y_right)


class MyDecisionTreeRegressor(MyDecisionTreeBase):
    def __init__(self, max_depth=5, min_samples_split=2, min_samples_leaf=1):
        super().__init__(max_depth, min_samples_split, min_samples_leaf)

    def _leaf_value(self, y):
        return np.mean(y)

    def _mse(self, y):
        if len(y) == 0:
            return 0.0
        return np.mean((y - np.mean(y)) ** 2)

    def _split_criterion(self, y_left, y_right):
        n = len(y_left) + len(y_right)
        return (len(y_left) / n) * self._mse(y_left) + (len(y_right) / n) * self._mse(y_right)


In [9]:
my_dt_clf = MyDecisionTreeClassifier(max_depth=5, min_samples_split=5, min_samples_leaf=2)
my_dt_clf.fit(X_train_m_enc, y_train_m)

y_pred_m_my_dt = my_dt_clf.predict(X_test_m_enc)

print("=== MyDecisionTreeClassifier (Mushrooms) ===")
print("Accuracy:", accuracy_score(y_test_m, y_pred_m_my_dt))
print("\nClassification report:")
print(classification_report(y_test_m, y_pred_m_my_dt))
print("\nConfusion matrix:")
print(confusion_matrix(y_test_m, y_pred_m_my_dt))


=== MyDecisionTreeClassifier (Mushrooms) ===
Accuracy: 0.9987692307692307

Classification report:
              precision    recall  f1-score   support

           e       1.00      1.00      1.00       842
           p       1.00      1.00      1.00       783

    accuracy                           1.00      1625
   macro avg       1.00      1.00      1.00      1625
weighted avg       1.00      1.00      1.00      1625


Confusion matrix:
[[842   0]
 [  2 781]]


In [10]:
my_dt_reg = MyDecisionTreeRegressor(max_depth=5, min_samples_split=5, min_samples_leaf=5)
my_dt_reg.fit(X_train_c_enc, y_train_c)

y_pred_c_my_dt = my_dt_reg.predict(X_test_c_enc)

mae_my_dt = mean_absolute_error(y_test_c, y_pred_c_my_dt)
rmse_my_dt = mean_squared_error(y_test_c, y_pred_c_my_dt)**0.5
r2_my_dt = r2_score(y_test_c, y_pred_c_my_dt)

print("=== MyDecisionTreeRegressor (Cars) ===")
print("MAE:", mae_my_dt)
print("RMSE:", rmse_my_dt)
print("R^2:", r2_my_dt)


=== MyDecisionTreeRegressor (Cars) ===
MAE: 2133.7195274674536
RMSE: 2730.337840199153
R^2: 0.727597818860648


## 4. Собственная реализация

1. **MyDecisionTreeClassifier**: Accuracy ~0.999. Работает корректно.
2. **MyDecisionTreeRegressor**: R2 ~0.73. Почти догнала улучшенный sklearn (0.75) и обогнала бейзлайн (0.63).


## 5. Выводы

| Модель | Задача | R2 / Accuracy |
|---|---|---|
| Tree Baseline | Reg | 0.628 |
| Tree Improved | Reg | 0.747 |
| My Tree | Reg | 0.728 |
| Tree (Any) | Class | ~1.0 |

Дерево требует настройки глубины для задач регрессии, иначе переобучается. Для простых задач классификации работает идеально "из коробки".
